In [1]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np


**Se cargan los installs, se convierte a fecha la columna created y se la usa como indice**

In [2]:
installs = pd.read_csv('data/installs.csv.gzip', low_memory=False, compression='gzip', parse_dates = ['created'], index_col=['created'])
installs = installs.sort_index()
installs.head()

,application_id,ref_type,ref_hash,click_hash,attributed,implicit,device_countrycode,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,ip_address,device_language
created,,,,,,,,,,,,,,,,,
2019-04-18 00:00:01.560,70,1891515180541284343,4432995619177048534,NaN,False,False,6287817205707153877,NaN,NaN,Apsalar-Postback,NaN,NaN,NaN,NaN,NaN,8409380583076227678,NaN
2019-04-18 00:00:01.851,70,1891515180541284343,5904733559638204455,NaN,False,False,6287817205707153877,NaN,NaN,Apsalar-Postback,NaN,NaN,NaN,NaN,NaN,806493034070542041,NaN
2019-04-18 00:00:05.152,65,1891515180541284343,896373747754111825,NaN,False,True,6287817205707153877,3.083059e+17,5.274185e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 9; SM-G9650 Bu...,8c8af5e3-96e7-4a49-9f17-cafa7f300f2c,af_app_opened,False,NaN,6319394158544413414,6.977049e+18
2019-04-18 00:00:05.589,27,1494519392962156891,3399210824535017892,NaN,False,False,6287817205707153877,NaN,6.794880e+18,http-kit/2.0,trivago/216 CFNetwork/978.0.7 Darwin/18.5.0,NaN,NaN,True,NaN,4206683315615624585,5.221863e+18
2019-04-18 00:00:06.795,339,1494519392962156891,1541425881979513687,NaN,False,False,6287817205707153877,NaN,6.794880e+18,http-kit/2.0,TikTok/109005 CFNetwork/758.5.3 Darwin/15.6.0,NaN,NaN,True,NaN,8335834010411202437,7.528974e+18


**Se agrupa por usuario y se queda con el primer install de la ventana. Luego se convierte a segundos.**

In [3]:
label1 = installs['2019-04-21':'2019-04-23'].reset_index().groupby('ref_hash').agg({'created' : 'min'}).reset_index()

In [4]:
label1['created'] = label1['created'] - pd.Timestamp('2019-04-21 00:00:00')
label1['created'] = label1['created'].transform(lambda x: x.total_seconds())

#### Ver esto, ¿deberia ser inner el join? Porque asi el modelo solo aprende de usuarios que van convertir

In [5]:
label1 = label1.merge(installs['2019-04-18':'2019-04-20'].reset_index()[["ref_hash"]],on="ref_hash", how="right")
label1.head(20)

,ref_hash,created
0,2493454867357963,19452.720
1,3589607969461633,108671.325
2,3589607969461633,108671.325
3,3861795021724487,252890.111
4,6455321535851721,217290.613
5,7944594974253876,220770.040
6,14664043499550181,97507.843
7,15057990565785689,238575.817
8,15057990565785689,238575.817
9,15695176139879422,239972.035


**A los que no aparecieron en la ventana de testing les asigno el valor maximo de tiempo: 3 dias(en segundos)**

In [6]:
label1["created"] = label1["created"].fillna(60*60*24*3)
label1 = label1.rename(columns={'created':'label'})

In [7]:
y=label1.loc[:,'label']
y


0          19452.720
1         108671.325
2         108671.325
3         252890.111
4         217290.613
5         220770.040
6          97507.843
7         238575.817
8         238575.817
9         239972.035
10        170757.389
11        243918.623
12         22624.206
13        225764.103
14        225764.103
15        107517.631
16        152442.672
17        152442.672
18         93014.879
19        245073.483
20        166275.213
21         11104.805
22         11104.805
23         11104.805
24          3211.751
25         48230.003
26         48230.003
27         48230.003
28        189169.026
29        103646.069
             ...    
160831    259200.000
160832    259200.000
160833    259200.000
160834    259200.000
160835    259200.000
160836    259200.000
160837    259200.000
160838    259200.000
160839    259200.000
160840    259200.000
160841    259200.000
160842    259200.000
160843    259200.000
160844    259200.000
160845    259200.000
160846    259200.000
160847    259

In [8]:
ventana_1 = installs['2019-04-18':'2019-04-20'].reset_index()

In [9]:
# data1=label1.drop('label',axis=1)
# data1.head()
#Aca se debe hacer limpieza de los datos... dejar features que sirven y hacer nuevos

In [10]:
ventana_1.head(5)

,created,application_id,ref_type,ref_hash,click_hash,attributed,implicit,device_countrycode,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,ip_address,device_language
0,2019-04-18 00:00:01.560,70,1891515180541284343,4432995619177048534,NaN,False,False,6287817205707153877,NaN,NaN,Apsalar-Postback,NaN,NaN,NaN,NaN,NaN,8409380583076227678,NaN
1,2019-04-18 00:00:01.851,70,1891515180541284343,5904733559638204455,NaN,False,False,6287817205707153877,NaN,NaN,Apsalar-Postback,NaN,NaN,NaN,NaN,NaN,806493034070542041,NaN
2,2019-04-18 00:00:05.152,65,1891515180541284343,896373747754111825,NaN,False,True,6287817205707153877,3.083059e+17,5.274185e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 9; SM-G9650 Bu...,8c8af5e3-96e7-4a49-9f17-cafa7f300f2c,af_app_opened,False,NaN,6319394158544413414,6.977049e+18
3,2019-04-18 00:00:05.589,27,1494519392962156891,3399210824535017892,NaN,False,False,6287817205707153877,NaN,6.794880e+18,http-kit/2.0,trivago/216 CFNetwork/978.0.7 Darwin/18.5.0,NaN,NaN,True,NaN,4206683315615624585,5.221863e+18
4,2019-04-18 00:00:06.795,339,1494519392962156891,1541425881979513687,NaN,False,False,6287817205707153877,NaN,6.794880e+18,http-kit/2.0,TikTok/109005 CFNetwork/758.5.3 Darwin/15.6.0,NaN,NaN,True,NaN,8335834010411202437,7.528974e+18


In [11]:
#Vamos a dejar las columnas que nos aporten informacion...

In [12]:
# Podemos "inventar" que los que no tienen info son falsos...
# data1['wifi']=data1['wifi'].fillna(False)

In [13]:
ventana_1.drop(['created','ref_type','click_hash','session_user_agent','user_agent','kind','trans_id','event_uuid', "device_brand", "device_model", "wifi", "device_language"],axis =1, inplace = True )


In [14]:
ventana_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160861 entries, 0 to 160860
Data columns (total 6 columns):
application_id        160861 non-null int64
ref_hash              160861 non-null int64
attributed            160861 non-null bool
implicit              160861 non-null bool
device_countrycode    160861 non-null int64
ip_address            160861 non-null int64
dtypes: bool(2), int64(4)
memory usage: 5.2 MB


In [15]:
# ventana_1["device_brand"] = ventana_1["device_brand"].fillna()

In [16]:
# data1=data1.fillna(0)

In [17]:
ventana_1.shape,y.shape

((160861, 6), (160861,))

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
    train_test_split(ventana_1, y, test_size=0.2, random_state=123)

In [19]:
reg=linear_model.LinearRegression()

In [20]:
reg.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [21]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((128688, 6), (32173, 6), (128688,), (32173,))

In [22]:
pred=reg.predict(X_test)

In [23]:
pred.max()

255212.4737655301

In [24]:
pred.shape

(32173,)

In [25]:
y_test

7523      259200.0
133848    259200.0
118915    259200.0
38555     259200.0
73812     259200.0
88516     259200.0
39632     259200.0
42551     259200.0
113608    259200.0
108540    259200.0
114020    259200.0
68813     259200.0
19831     259200.0
62463     259200.0
85352     259200.0
12120     259200.0
25390     259200.0
123132    259200.0
147773    259200.0
62030     259200.0
122947    259200.0
69645     259200.0
63995     259200.0
14476     259200.0
86613     259200.0
55899     259200.0
116418    259200.0
36702     259200.0
74124     259200.0
107964    259200.0
            ...   
47990     259200.0
140962    259200.0
140955    259200.0
147706    259200.0
47921     259200.0
133627    259200.0
75534     259200.0
65809     259200.0
84439     259200.0
63354     259200.0
65464     259200.0
78813     259200.0
160070    259200.0
82021     259200.0
29127     259200.0
139598    259200.0
78365     259200.0
103806    259200.0
144686    259200.0
18325     259200.0
88260     259200.0
35552     25

Square mean error, es la que se usa para calcular el error en kagle. Da mas bonito asi.

In [26]:
# Square mean square error
mean_squared_error(y_test,pred)**0.5
# np.mean((pred-y_test)**2)

28461.64078441799

#Ahora estas predicciones las agregamos como feature para la segunda ventana...

Empezamos con los nuevos labels


In [44]:
label2 = installs['2019-04-24':'2019-04-26'].reset_index().groupby('ref_hash').agg({'created' : 'min'})

In [45]:
label2['created'] = label2['created'] - pd.Timestamp('2019-04-24 00:00:00')
label2['created'] = label2['created'].transform(lambda x: x.total_seconds())

In [46]:
label2 = label2.reset_index()
label2 = label2.merge(installs['2019-04-21':'2019-04-23'].reset_index(),on="ref_hash", how="right")
label2 = label2.rename(columns={'created_x':'label','created_y':'created'})
label2

,ref_hash,label,created,application_id,ref_type,click_hash,attributed,implicit,device_countrycode,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,ip_address,device_language
0,622102439689666,979.059,2019-04-23 23:17:02.399,121,1891515180541284343,NaN,False,True,6287817205707153877,2.523246e+18,5.151882e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 6.0.1; LG-K200...,f8b3f19d-c761-4d6a-9f20-4e1832d38146,Open,True,NaN,1848802948798920304,6.977049e+18
1,622102439689666,979.059,2019-04-23 23:17:02.787,121,1891515180541284343,NaN,False,False,6287817205707153877,2.523246e+18,5.151882e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 6.0.1; LG-K200...,NaN,NaN,True,NaN,1848802948798920304,6.977049e+18
2,928188758790177,247212.597,2019-04-21 01:50:16.400,68,1494519392962156891,NaN,False,False,6287817205707153877,NaN,6.794880e+18,http-kit/2.0,Grability/17147 CFNetwork/978.0.7 Darwin/18.5.0,NaN,NaN,True,NaN,3483843913495440783,8.441417e+18
3,4129436938177104,229521.524,2019-04-21 20:56:18.807,117,1891515180541284343,NaN,False,False,6287817205707153877,NaN,2.399584e+18,adjust.com,NaN,NaN,NaN,NaN,NaN,5840064274212175308,3.301378e+18
4,5133475695351500,209187.715,2019-04-22 23:38:06.589,194,1494519392962156891,NaN,False,False,6287817205707153877,NaN,6.794880e+18,http-kit/2.0,Gardenscapes/3.3.2 CFNetwork/978.0.7 Darwin/18...,NaN,NaN,True,NaN,2437566759560000641,8.441417e+18
5,7881267125156197,176402.652,2019-04-22 14:15:28.970,210,1891515180541284343,NaN,False,False,6287817205707153877,NaN,8.479522e+18,http-kit/2.0,Mozilla/5.0 (Linux; Android 9; SM-G970F Build/...,NaN,NaN,NaN,NaN,97224138037576704,3.301378e+18
6,7944594974253876,9055.796,2019-04-23 13:19:30.040,281,1891515180541284343,NaN,False,False,6287817205707153877,NaN,NaN,Dalvik/2.1.0 (Linux; U; Android 9; SM-N960U1 B...,NaN,NaN,NaN,NaN,NaN,4063933476320529056,NaN
7,13912050945871915,52090.338,2019-04-23 12:54:17.555,226,1891515180541284343,NaN,False,False,6287817205707153877,4.567867e+18,1.292000e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 8.0.0; BLADE V...,NaN,NaN,False,NaN,3645808844944603776,6.977049e+18
8,24294411704078426,182902.601,2019-04-23 03:02:23.309,36,1891515180541284343,NaN,False,False,6287817205707153877,3.083059e+17,9.186120e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 6.0.1; SM-G532...,NaN,NaN,True,NaN,4950519528220185340,6.977049e+18
9,24625906561621678,225275.348,2019-04-22 15:08:28.648,226,1891515180541284343,NaN,False,False,6287817205707153877,2.208835e+18,4.073186e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 8.1.0; moto e5...,NaN,NaN,False,NaN,653240972862608991,6.977049e+18


In [47]:
y2=label2.loc[:,'label']


In [48]:
data2=label2.drop('label',axis=1)
data2.head()

,ref_hash,created,application_id,ref_type,click_hash,attributed,implicit,device_countrycode,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,ip_address,device_language
0,622102439689666,2019-04-23 23:17:02.399,121,1891515180541284343,NaN,False,True,6287817205707153877,2.523246e+18,5.151882e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 6.0.1; LG-K200...,f8b3f19d-c761-4d6a-9f20-4e1832d38146,Open,True,NaN,1848802948798920304,6.977049e+18
1,622102439689666,2019-04-23 23:17:02.787,121,1891515180541284343,NaN,False,False,6287817205707153877,2.523246e+18,5.151882e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 6.0.1; LG-K200...,NaN,NaN,True,NaN,1848802948798920304,6.977049e+18
2,928188758790177,2019-04-21 01:50:16.400,68,1494519392962156891,NaN,False,False,6287817205707153877,NaN,6.794880e+18,http-kit/2.0,Grability/17147 CFNetwork/978.0.7 Darwin/18.5.0,NaN,NaN,True,NaN,3483843913495440783,8.441417e+18
3,4129436938177104,2019-04-21 20:56:18.807,117,1891515180541284343,NaN,False,False,6287817205707153877,NaN,2.399584e+18,adjust.com,NaN,NaN,NaN,NaN,NaN,5840064274212175308,3.301378e+18
4,5133475695351500,2019-04-22 23:38:06.589,194,1494519392962156891,NaN,False,False,6287817205707153877,NaN,6.794880e+18,http-kit/2.0,Gardenscapes/3.3.2 CFNetwork/978.0.7 Darwin/18...,NaN,NaN,True,NaN,2437566759560000641,8.441417e+18


In [50]:
#Esas pred se las tenemos que agregar al set de entrenamiento ...
#¿Como hacemos? solo tenemos 990 filas! rellenamos con algun numero? o sea 999999?
#Tambien podemos usar el label de la anterior... para esta
data2['label1']=y
data2.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 160617 entries, 0 to 160616
Data columns (total 19 columns):
ref_hash              160617 non-null int64
created               160617 non-null datetime64[ns]
application_id        160617 non-null int64
ref_type              160617 non-null int64
click_hash            312 non-null object
attributed            160617 non-null bool
implicit              160617 non-null bool
device_countrycode    160617 non-null int64
device_brand          92135 non-null float64
device_model          151369 non-null float64
session_user_agent    156727 non-null object
user_agent            110182 non-null object
event_uuid            34677 non-null object
kind                  34677 non-null object
wifi                  98105 non-null object
trans_id              2847 non-null object
ip_address            160617 non-null int64
device_language       151440 non-null float64
label1                160617 non-null float64
dtypes: bool(2), datetime64[ns](1), floa

In [51]:
data2.drop(['created','ref_type','click_hash','session_user_agent','user_agent','kind','trans_id','event_uuid'],axis =1, inplace = True )
data2.fillna(0)


,ref_hash,application_id,attributed,implicit,device_countrycode,device_brand,device_model,wifi,ip_address,device_language,label1
0,622102439689666,121,False,True,6287817205707153877,2.523246e+18,5.151882e+18,True,1848802948798920304,6.977049e+18,19452.720
1,622102439689666,121,False,False,6287817205707153877,2.523246e+18,5.151882e+18,True,1848802948798920304,6.977049e+18,108671.325
2,928188758790177,68,False,False,6287817205707153877,0.000000e+00,6.794880e+18,True,3483843913495440783,8.441417e+18,108671.325
3,4129436938177104,117,False,False,6287817205707153877,0.000000e+00,2.399584e+18,0,5840064274212175308,3.301378e+18,252890.111
4,5133475695351500,194,False,False,6287817205707153877,0.000000e+00,6.794880e+18,True,2437566759560000641,8.441417e+18,217290.613
5,7881267125156197,210,False,False,6287817205707153877,0.000000e+00,8.479522e+18,0,97224138037576704,3.301378e+18,220770.040
6,7944594974253876,281,False,False,6287817205707153877,0.000000e+00,0.000000e+00,0,4063933476320529056,0.000000e+00,97507.843
7,13912050945871915,226,False,False,6287817205707153877,4.567867e+18,1.292000e+18,False,3645808844944603776,6.977049e+18,238575.817
8,24294411704078426,36,False,False,6287817205707153877,3.083059e+17,9.186120e+18,True,4950519528220185340,6.977049e+18,238575.817
9,24625906561621678,226,False,False,6287817205707153877,2.208835e+18,4.073186e+18,False,653240972862608991,6.977049e+18,239972.035


In [52]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 160617 entries, 0 to 160616
Data columns (total 11 columns):
ref_hash              160617 non-null int64
application_id        160617 non-null int64
attributed            160617 non-null bool
implicit              160617 non-null bool
device_countrycode    160617 non-null int64
device_brand          92135 non-null float64
device_model          151369 non-null float64
wifi                  98105 non-null object
ip_address            160617 non-null int64
device_language       151440 non-null float64
label1                160617 non-null float64
dtypes: bool(2), float64(4), int64(4), object(1)
memory usage: 12.6+ MB


In [55]:
data2['wifi']=data2['wifi'].fillna(False)

In [56]:
data2.shape,y2.shape

((160617, 11), (160617,))

In [57]:
X2_train, X2_test, y2_train, y2_test = \
    train_test_split(data2, y2, test_size=0.2, random_state=123)

In [58]:
reg2=linear_model.LinearRegression()


In [73]:
X2_train=X2_train.fillna(0)
y2_train=y2_train.fillna(60*60*24*3)
X2_test=X2_test.fillna(0)
y2_test=y2_test.fillna(60*60*24*3)


In [74]:
reg2.fit(X2_train,y2_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [75]:
X2_train.shape,X2_test.shape,y2_train.shape,y2_test.shape


((128493, 11), (32124, 11), (128493,), (32124,))

In [76]:
pred2=reg2.predict(X2_test)


In [77]:
mean_squared_error(y2_test,pred2)**0.5


16972.09292009813